need to install ffmpeg and add the path to your environment

``` sh
https://www.gyan.dev/ffmpeg/builds/#release-builds
```
``` sh
pip install ffmpeg-python
```

In [25]:
!pip install ffmpeg-python

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
Using cached future-1.0.0-py3-none-any.whl (491 kB)


In [ ]:
# Convert videos to MP4 using FFmpeg 

import os
import subprocess

def is_ffmpeg_installed():
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        return True
    except FileNotFoundError:
        return False

def convert_videos_to_mp4(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".mp4")
        
        if os.path.isfile(input_path):
            if os.path.exists(output_path):
                print(f"Skipping: {filename} (already converted)")
                continue
            
            if filename.lower().endswith(".mp4"):
                try:
                    os.rename(input_path, output_path)
                    print(f"Copied: {filename} -> {output_path}")
                except Exception as e:
                    print(f"Error moving {filename}: {e}")
            else:
                # command = [
                #     "ffmpeg", "-hwaccel", "cuda", "-i", input_path,
                #     "-c:v", "h264_nvenc", "-preset", "fast", "-b:v", "5000k",
                #     "-pix_fmt", "yuv420p",
                #     "-c:a", "aac", "-b:a", "128k",
                #     output_path
                # ]
                
                command = [
                    "ffmpeg", "-hwaccel", "cuda", "-i", input_path,
                    "-c:v", "copy",  # Copy video without re-encoding
                    "-c:a", "copy",  # Copy audio without re-encoding
                    "-movflags", "+faststart",  # Optimize MP4 for streaming
                    output_path
                ]
                
                try:
                    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, check=True)
                    print(f"Converted: {filename} -> {output_path}")
                except FileNotFoundError:
                    print("Error: FFmpeg is not installed or not in system PATH.")
                except subprocess.CalledProcessError as e:
                    print(f"Error converting {filename}: {e.stderr.decode(errors='ignore')}")
                except Exception as e:
                    print(f"Unexpected error processing {filename}: {e}")

if __name__ == "__main__":
    base_input_folder = "Recordings" 
    base_output_folder = "converted_videos" 
    
    if not os.path.exists(base_input_folder):
        print(f"Error: Input folder {base_input_folder} does not exist.")
    else:
        subfolders = [os.path.join(base_input_folder, d) for d in os.listdir(base_input_folder) if os.path.isdir(os.path.join(base_input_folder, d))]
        
        if subfolders:
            for input_folder in subfolders:
                relative_subfolder = os.path.relpath(input_folder, base_input_folder)
                output_folder = os.path.join(base_output_folder, relative_subfolder)
                convert_videos_to_mp4(input_folder, output_folder)
        else:
            convert_videos_to_mp4(base_input_folder, base_output_folder)


Skipping: IMG_4917.MOV (already converted)
Skipping: IMG_4918-001.MOV (already converted)
Copied: Part1.mp4 -> converted_videos\Part1.mp4
Skipping: Part2-003.mp4 (already converted)
Skipping: Recording from 2025-03-03 09-11-03.688254.webm (already converted)
Skipping: Recording from 2025-03-03 10-51-23.391532.webm (already converted)
Skipping: WIN_20250303_09_11_30_Pro.mp4 (already converted)
Skipping: WIN_20250303_09_11_53_Pro.mp4 (already converted)
Skipping: WIN_20250303_09_16_32_Pro.mp4 (already converted)
Skipping: WIN_20250303_09_18_49_Pro.mp4 (already converted)
Skipping: WIN_20250303_09_19_13_Pro-002.mp4 (already converted)
Skipping: WIN_20250303_09_34_26_Pro(1)-011.mp4 (already converted)
Skipping: WIN_20250303_09_34_26_Pro-012.mp4 (already converted)
Skipping: WIN_20250303_10_47_26_Pro.mp4 (already converted)
Skipping: WIN_20250303_10_53_26_Pro(1)-009.mp4 (already converted)
Skipping: WIN_20250303_10_53_26_Pro-010.mp4 (already converted)
Skipping: WIN_20250303_10_53_46_Pro.mp

In [1]:
# Reduce video size using FFmpeg

import os
import subprocess

def reduce_video_size(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        if os.path.isfile(input_path) and filename.lower().endswith(".mp4"):
            output_path = os.path.join(output_folder, filename)
            
            if os.path.exists(output_path):
                print(f"Skipping: {filename} already exists in {output_folder}")
                continue
            
            # command = [
            #     "ffmpeg", "-y", "-hwaccel", "cuda", "-hwaccel_output_format", "cuda", "-i", input_path,
            #     "-c:v", "hevc_nvenc", "-preset", "slow", "-b:v", "200k", 
            #     "-maxrate", "200k", "-bufsize", "400k", "-cq", "35", 
            #     # "-c:a", "aac", "-b:a", "64k", # Reduce audio bitrate
            #     "-an", # Disable audio
            #     output_path
            # ]
            
            command = [
                "ffmpeg", "-y", "-hwaccel", "cuda", "-hwaccel_output_format", "cuda", "-i", input_path,
                "-c:v", "hevc_nvenc", "-preset", "slow", "-cq", "28",  # Lower CQ = better quality (0-51, lower is better)
                "-b:v", "0",  # Ignore bitrate limit, use only CQ for better compression
                "-an",  # Disable audio
                output_path
            ]

            
            try:
                subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, check=True)
                print(f"Reduced size: {filename} -> {output_path}")
            except FileNotFoundError:
                print("Error: FFmpeg is not installed or not in system PATH.")
            except subprocess.CalledProcessError as e:
                print(f"Error reducing size of {filename}: {e.stderr.decode(errors='ignore')}")
            except Exception as e:
                print(f"Unexpected error processing {filename}: {e}")

if __name__ == "__main__":
    base_input_folder = "D:\Recordings\converted_videos - Copy" 
    base_output_folder = "reduced_videos2" 
    
    if not os.path.exists(base_input_folder):
        print(f"Error: Input folder {base_input_folder} does not exist.")
    else:
        subfolders = [os.path.join(base_input_folder, d) for d in os.listdir(base_input_folder) if os.path.isdir(os.path.join(base_input_folder, d))]
        
        if subfolders:
            for input_folder in subfolders:
                relative_subfolder = os.path.relpath(input_folder, base_input_folder)
                output_folder = os.path.join(base_output_folder, relative_subfolder)
                reduce_video_size(input_folder, output_folder)
        reduce_video_size(base_input_folder, base_output_folder)


Skipping: Part1.mp4 already exists in reduced_videos2
Skipping: Part2-003.mp4 already exists in reduced_videos2
Skipping: Recording from 2025-03-03 09-11-03.688254.mp4 already exists in reduced_videos2
Skipping: Recording from 2025-03-03 10-51-23.391532.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_09_11_30_Pro.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_09_11_53_Pro.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_09_16_32_Pro.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_09_18_49_Pro.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_09_19_13_Pro-002.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_09_34_26_Pro(1)-011.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_09_34_26_Pro-012.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_10_47_26_Pro.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_10_47_36_Pro.mp4 already exists in reduced_videos2
Skipping: WIN_20250303_10_4

In [1]:
# Rename videos in a folder
# Warning: This script will rename all videos in the folder to "Video 01.mp4", "Video 02.mp
# The original filenames will be lost
# Make sure to backup the original files before running this script

import os

def rename_videos(folder_path):
    videos = [f for f in os.listdir(folder_path) if f.lower().endswith(".mp4")]
    videos.sort()  # Ensure the files are renamed in order
    
    for index, video in enumerate(videos, start=1):
        new_name = f"Video {index:02d}.mp4"
        old_path = os.path.join(folder_path, video)
        new_path = os.path.join(folder_path, new_name)
        
        os.rename(old_path, new_path)
        print(f"Renamed: {video} -> {new_name}")

folder_path = "reduced_videos2"  
rename_videos(folder_path)


Renamed: Part1.mp4 -> Video 01.mp4
Renamed: Part2-003.mp4 -> Video 02.mp4
Renamed: Recording from 2025-03-03 09-11-03.688254.mp4 -> Video 03.mp4
Renamed: Recording from 2025-03-03 10-51-23.391532.mp4 -> Video 04.mp4
Renamed: WIN_20250303_09_11_30_Pro.mp4 -> Video 05.mp4
Renamed: WIN_20250303_09_11_53_Pro.mp4 -> Video 06.mp4
Renamed: WIN_20250303_09_16_32_Pro.mp4 -> Video 07.mp4
Renamed: WIN_20250303_09_18_49_Pro.mp4 -> Video 08.mp4
Renamed: WIN_20250303_09_19_13_Pro-002.mp4 -> Video 09.mp4
Renamed: WIN_20250303_09_34_26_Pro(1)-011.mp4 -> Video 10.mp4
Renamed: WIN_20250303_09_34_26_Pro-012.mp4 -> Video 11.mp4
Renamed: WIN_20250303_10_47_26_Pro.mp4 -> Video 12.mp4
Renamed: WIN_20250303_10_47_36_Pro.mp4 -> Video 13.mp4
Renamed: WIN_20250303_10_47_46_Pro.mp4 -> Video 14.mp4
Renamed: WIN_20250303_10_53_26_Pro(1)-009.mp4 -> Video 15.mp4
Renamed: WIN_20250303_10_53_26_Pro-010.mp4 -> Video 16.mp4
Renamed: WIN_20250303_10_53_46_Pro.mp4 -> Video 17.mp4
Renamed: WIN_20250303_10_54_21_Pro-008.mp4 

In [ ]:
# Split videos into 15-second segments and save metadata in a CSV file

import os
import ffmpeg
import pandas as pd
from datetime import timedelta

def split_videos(input_folder, output_folder, segment_length=15):
    os.makedirs(output_folder, exist_ok=True)
    csv_data = []
    
    for file in os.listdir(input_folder):
        if file.endswith(".mp4"):
            video_path = os.path.join(input_folder, file)
            
            # Get video duration
            probe = ffmpeg.probe(video_path)
            duration = float(probe['format']['duration'])
            
            base_name = os.path.splitext(file)[0]
            segment_count = 1
            
            for start_time in range(0, int(duration), segment_length):
                end_time = min(start_time + segment_length, duration)
                split_name = f"{base_name}-{segment_count:03d}.mp4"
                output_path = os.path.join(output_folder, split_name)
                
                # Split the video
                ffmpeg.input(video_path, ss=start_time, t=segment_length).output(output_path, c="h264_nvenc").run(overwrite_output=True, quiet=True)
                
                # Save metadata for CSV
                csv_data.append([file, split_name, str(timedelta(seconds=start_time)), str(timedelta(seconds=end_time))])
                
                segment_count += 1
    
    # Save CSV
    csv_path = os.path.join(output_folder, "video_segments.csv")
    df = pd.DataFrame(csv_data, columns=["video_name", "split_video", "start_time", "end_time"])
    df.to_csv(csv_path, index=False)
    print(f"Processing complete! CSV saved at: {csv_path}")

split_videos("reduced_videos2", "D:\Recordings\Splitted_Videos")


Processing complete! CSV saved at: D:\Recordings\Splitted_Videos\video_segments.csv


In [5]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     -------------

In [6]:
import torch

print(torch.__version__)


2.4.1+cu124


In [ ]:
import cv2
import numpy as np
import os
import torch
import torchvision.transforms as T

def adjust_contrast(image, alpha, beta):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Convert image to float32 and normalize to [0,1]
    image_tensor = torch.from_numpy(image).float().to(device) / 255.0
    
    # Apply contrast and brightness adjustment
    adjusted_tensor = torch.clamp(image_tensor * alpha + beta / 255.0, 0, 1)
    
    # Convert back to uint8 and return to CPU
    return (adjusted_tensor.cpu().numpy() * 255).astype(np.uint8)

def process_videos(input_folder, output_folder, alpha=1.5, beta=0):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            
            cap = cv2.VideoCapture(input_path)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                adjusted_frame = adjust_contrast(frame, alpha, beta)
                out.write(adjusted_frame)
            
            cap.release()
            out.release()
            print(f"Processed: {filename}")
    
    print("Processing complete.")

input_folder = "13"
output_folder = "output_13"
process_videos(input_folder, output_folder, alpha=1.8, beta=30) # Alpha = contrast, Beta = brightness

Processed: Video 15-028.mp4
Processed: Video 15-029.mp4
Processed: Video 15-030.mp4
Processed: Video 15-031.mp4
Processed: Video 15-032.mp4
Processed: Video 15-033.mp4
Processed: Video 15-034.mp4
Processed: Video 15-035.mp4
Processed: Video 15-036.mp4
Processed: Video 15-037.mp4
Processed: Video 15-038.mp4
Processed: Video 15-039.mp4
Processed: Video 15-040.mp4
Processed: Video 15-041.mp4
Processed: Video 15-042.mp4
Processed: Video 15-043.mp4
Processed: Video 15-044.mp4
Processed: Video 15-045.mp4
Processed: Video 15-046.mp4
Processed: Video 15-047.mp4
Processed: Video 15-048.mp4
Processed: Video 15-049.mp4
Processed: Video 15-050.mp4
Processed: Video 15-051.mp4
Processed: Video 15-052.mp4
Processed: Video 15-053.mp4
Processed: Video 15-054.mp4
Processed: Video 15-055.mp4
Processed: Video 15-056.mp4
Processed: Video 15-057.mp4
Processed: Video 15-058.mp4
Processed: Video 15-059.mp4
Processed: Video 15-060.mp4
Processed: Video 15-061.mp4
Processed: Video 15-062.mp4
Processed: Video 15-

In [ ]:
#Rename the videos (has subfolders) with the folder name

import os

def rename_videos_with_folder_name(base_folder):
    for subfolder in os.listdir(base_folder):
        subfolder_path = os.path.join(base_folder, subfolder)
        if os.path.isdir(subfolder_path):
            for idx, filename in enumerate(os.listdir(subfolder_path)):
                input_path = os.path.join(subfolder_path, filename)
                if os.path.isfile(input_path) and filename.lower().endswith(".mp4"):
                    new_filename = f"{subfolder}{idx+1:03d}.mp4"
                    new_path = os.path.join(subfolder_path, new_filename)
                    os.rename(input_path, new_path)
                    print(f"Renamed: {filename} -> {new_filename}")

if __name__ == "__main__":
    base_folder = "reduced_videos2" 
    rename_videos_with_folder_name(base_folder)

Renamed: 37(side).mp4 -> Grazing001.mp4
Renamed: 38(front).mp4 -> Grazing002.mp4
Renamed: 38(side).mp4 -> Grazing003.mp4
Renamed: 39(front).mp4 -> Grazing004.mp4
Renamed: 39(side).mp4 -> Grazing005.mp4
Renamed: 038 (run side).mp4 -> Running001.mp4
Renamed: 039 (run side).mp4 -> Running002.mp4
Renamed: 040 (run side).mp4 -> Running003.mp4
Renamed: 041 (run side).mp4 -> Running004.mp4
Renamed: IMG_5483(0).mp4 -> Running005.mp4
Renamed: 025(sit side).mp4 -> Sitting001.mp4
Renamed: 027(sit front).mp4 -> Sitting002.mp4
Renamed: 028(sit front).mp4 -> Sitting003.mp4
Renamed: 032(sit front).mp4 -> Sitting004.mp4
Renamed: IMG_5320.mp4 -> Sitting005.mp4


In [1]:
# Make csv for labels (has sub folders)

import os
import csv

def label_dataset(root_dir, output_csv):
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["video_name", "tag"])
        
        for category in os.listdir(root_dir):
            category_path = os.path.join(root_dir, category)
            if os.path.isdir(category_path):
                for video in os.listdir(category_path):
                    video_path = os.path.join(category, video)  # Relative path
                    writer.writerow([video_path, category])

if __name__ == "__main__":
    dataset_folder = "Grazing_Running_Sitting Sheep_Classes"  # Folder containing the videos
    output_file = "Grazing_Running_Sitting Sheep_Classes/train.csv"
    label_dataset(dataset_folder, output_file)
    print(f"CSV file '{output_file}' created successfully.")


CSV file 'Grazing_Running_Sitting Sheep_Classes/train.csv' created successfully.


In [1]:
# Move out the videos from subfolders to main folder

import os
import shutil

def move_videos_to_main_folder(base_folder):
    for subfolder in os.listdir(base_folder):
        subfolder_path = os.path.join(base_folder, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                input_path = os.path.join(subfolder_path, filename)
                if os.path.isfile(input_path) and filename.lower().endswith(".mp4"):
                    new_path = os.path.join(base_folder, filename)
                    shutil.move(input_path, new_path)
                    print(f"Moved: {filename} -> {base_folder}")

if __name__ == "__main__":
    base_folder = "D:\Recordings\Cleaned Videos\Video Split - Copy"  # Base folder containing the videos
    move_videos_to_main_folder(base_folder)

Moved: Video 01-001.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-002.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-003.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-005.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-006.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-007.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-008.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-009.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-010.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-011.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-012.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-014.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-015.mp4 -> D:\Recordings\Cleaned Videos\Video Split - Copy
Moved: Video 01-016.mp4 -

In [24]:
# Make csv for labels (no sub folders)

import os
import csv

# Folder containing the video files
folder_path = "SheepTest copy"  # Change this to your actual folder path
output_csv = "SheepTest copy/test.csv"

# Get all video files in the folder
video_files = [f for f in os.listdir(folder_path) if f.endswith(".mp4")]

# Extract tags from filenames
data = []
for video in video_files:
    tag = "".join(filter(str.isalpha, video.rsplit(".", 1)[0]))  # Extract the alphabetic part as tag
    data.append([video, tag])

# Write to CSV
with open(output_csv, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["video_name", "tag"])
    writer.writerows(data)

print(f"CSV file '{output_csv}' has been created.")


CSV file 'SheepTest copy/test.csv' has been created.


In [3]:
import os
import random
import csv
import subprocess

# Define test set directory
test_dir = r"Goat\Test"

# Get all video files
video_files = []
for root, _, files in os.walk(test_dir):
    for file in files:
        if file.endswith(".mp4"):
            video_files.append(os.path.join(root, file))

# Shuffle video order randomly
random.shuffle(video_files)

# Create a text file list for ffmpeg
ffmpeg_list_file = "video_list.txt"
with open(ffmpeg_list_file, "w") as f:
    for video in video_files:
        f.write(f"file '{video}'\n")

# Generate CSV file
csv_file = "stitched_videos.csv"
start_time = 0
with open(csv_file, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["video_path", "start_time", "end_time"])
    
    for video in video_files:
        duration_cmd = f'ffprobe -i "{video}" -show_entries format=duration -v quiet -of csv="p=0"'
        duration = float(subprocess.check_output(duration_cmd, shell=True).decode().strip())

        end_time = start_time + duration
        csv_writer.writerow([video, f"{start_time:.2f}", f"{end_time:.2f}"])
        start_time = end_time

# Use ffmpeg to stitch videos together
output_video = "stitched_output.mp4"
concat_cmd = f'ffmpeg -f concat -safe 0 -i {ffmpeg_list_file} -c copy {output_video}'
subprocess.run(concat_cmd, shell=True)

print(f"Stitched video saved as {output_video}")
print(f"CSV metadata saved as {csv_file}")


Stitched video saved as stitched_output.mp4
CSV metadata saved as stitched_videos.csv


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import time

def predict_real_time(video_path, model, encoder, window_size=30, stride=8, img_size=(64, 64), interval=5):
    """
    Predicts actions from a video stream in real-time every `interval` seconds.

    Args:
        video_path (str): Path to the video file.
        model (tf.keras.Model): Trained 3D CNN model.
        encoder (LabelEncoder): Encoder for decoding class labels.
        window_size (int): Number of frames per prediction window.
        stride (int): Number of frames to move between windows.
        img_size (tuple): Resize dimensions for frames.
        interval (int): Time interval (in seconds) for predictions.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_buffer = []
    frame_idx = 0
    last_pred_time = time.time()
    
    print(f"Processing video: {video_path}")
    print(f"Total frames: {total_frames}, FPS: {fps}")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize and normalize frame
        frame = cv2.resize(frame, img_size)
        frame = frame / 255.0
        frame_buffer.append(frame)
        
        # Predict every `interval` seconds
        if time.time() - last_pred_time >= interval:
            if len(frame_buffer) >= window_size:
                input_data = np.expand_dims(np.array(frame_buffer[-window_size:]), axis=0)  # Last `window_size` frames
                pred = model.predict(input_data, verbose=0)
                class_idx = np.argmax(pred)
                predicted_label = encoder.inverse_transform([class_idx])[0]

                # Display prediction with timestamp
                timestamp = frame_idx / fps
                minutes = int(timestamp // 60)
                seconds = int(timestamp % 60)
                print(f"[{minutes:02d}:{seconds:02d}] Predicted Action: {predicted_label}")

                last_pred_time = time.time()
        
        frame_idx += 1
    
    cap.release()

# Load saved model
model = tf.keras.models.load_model("goat_action_3dcnn-30frames.h5")

# Initialize encoder with class labels
CLASSES = ['Running', 'Sitting', 'Grazing']
encoder = LabelEncoder()
encoder.fit(CLASSES)

# Path to long video
video_path = r"stitched_output.mp4"

# Predict actions in real-time every 5 seconds
predict_real_time(video_path, model, encoder, window_size=30, stride=8, img_size=(64, 64), interval=5)

Processing video: stitched_output.mp4
Total frames: 4540, FPS: 41.91179785039039
[00:06] Predicted Action: Grazing
[00:13] Predicted Action: Grazing
[00:20] Predicted Action: Grazing
[00:28] Predicted Action: Grazing
[00:35] Predicted Action: Grazing
[00:43] Predicted Action: Grazing
[00:50] Predicted Action: Grazing
[00:56] Predicted Action: Running
[01:02] Predicted Action: Sitting
[01:08] Predicted Action: Sitting
[01:13] Predicted Action: Sitting
[01:20] Predicted Action: Grazing
[01:27] Predicted Action: Grazing
[01:34] Predicted Action: Grazing
[01:41] Predicted Action: Grazing


In [2]:
# Count the number of videos in a folder and its subfolders

import os

def count_videos_in_folder(base_folder):
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv')
    video_count = 0
    
    for root, _, files in os.walk(base_folder):
        for file in files:
            if file.lower().endswith(video_extensions):
                video_count += 1
    
    return video_count

base_folder = r"D:\Recordings\Cleaned Videos\Segmented Videos" 
total_videos = count_videos_in_folder(base_folder)
print(f"Total number of videos: {total_videos}")

Total number of videos: 3470


In [4]:
import os

def count_videos_in_subfolders(base_folder):
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv')
    subfolder_video_counts = {}
    
    for subfolder in os.listdir(base_folder):
        subfolder_path = os.path.join(base_folder, subfolder)
        if os.path.isdir(subfolder_path):
            video_count = 0
            for root, _, files in os.walk(subfolder_path):
                for file in files:
                    if file.lower().endswith(video_extensions):
                        video_count += 1
            subfolder_video_counts[subfolder] = video_count
    
    return subfolder_video_counts

base_folder = r"C:\Users\Raymond\Downloads\Segmented Videos - Maam Jas"  # Change this to your actual folder path
video_counts = count_videos_in_subfolders(base_folder)
for subfolder, count in video_counts.items():
    print(f"{subfolder}: {count} videos")

Module 1: 156 videos
Module 10: 174 videos
Module 11: 184 videos
Module 12: 176 videos
Module 13: 165 videos
Module 14: 164 videos
Module 15: 193 videos
Module 16: 160 videos
Module 17: 208 videos
Module 18: 201 videos
Module 19: 209 videos
Module 2: 69 videos
Module 20: 180 videos
Module 3: 148 videos
Module 4: 170 videos
Module 5: 187 videos
module 6: 209 videos
Module 7: 185 videos
Module 8: 158 videos
Module 9: 174 videos
